In [32]:
import numpy as np
import pandas as pd
from scipy.stats import t as student_t
from scipy.optimize import minimize

In [33]:
def t_regression(csv_file):
    df = pd.read_csv(csv_file).dropna()
    X = df[["x1", "x2", "x3"]].to_numpy(dtype=float)
    y = df["y"].to_numpy(dtype=float)

    n = len(y)
    Z = np.column_stack([np.ones(n), X])

    beta_init, *_ = np.linalg.lstsq(Z, y, rcond=None)
    residuals = y - Z @ beta_init
    sigma_init = np.std(residuals, ddof=1)
    if not np.isfinite(sigma_init) or sigma_init <= 0:
        sigma_init = 1.0
    nu_init = 10.0

    x0 = np.concatenate([beta_init, [np.log(sigma_init)], [np.log(nu_init - 2.0)]])
    
    def negative_log_likelihood(theta):
        beta = theta[:4]
        sigma = np.exp(theta[4])
        nu = np.exp(theta[5]) + 2

        mu_hat = Z @ beta
        log_likelihood =student_t.logpdf(y, df=nu, loc=mu_hat, scale=sigma)
        if not np.all(np.isfinite(log_likelihood)):
            return 1e100
        return -float(np.sum(log_likelihood))

    result = minimize(negative_log_likelihood, x0, method="L-BFGS-B", options={"maxiter": 5000})
    theta = result.x
    beta = theta[:4]
    sigma = float(np.exp(theta[4]))
    nu = float(2 + np.exp(theta[5]))

    printout = [0.0, sigma, nu, beta[0], beta[1], beta[2], beta[3]]
    return printout

In [35]:
parameters = t_regression("test7_3.csv")
print(parameters)

[0.0, 0.04854800092813226, 4.598272329295343, np.float64(0.04263420866945536), np.float64(0.9748781699653598), np.float64(2.041192179123082), np.float64(3.1548046466224933)]
